In [1]:
import chromadb
from chromadb.config import Settings
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough

from typing import List
from typing_extensions import Annotated, TypedDict
import json

from langchain_ollama import ChatOllama

from langchain.load import dumps, loads

from operator import itemgetter



In [2]:
chroma_client = chromadb.HttpClient(host="127.0.0.1", port = 8000,
                                    settings=Settings(allow_reset=True,
                                                      anonymized_telemetry=False))

In [3]:
local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

Indexado: solo correr para indexar algo nuevo

In [4]:
# # Generar contexto, splitear y cargar en la base de datos

# from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# # data = loader.load()
# # loader = PyPDFLoader("DD-CAREM25CO-3-r0_b.pdf")
# loader = PyPDFLoader("MD-CAREM25XT-12-r2.pdf")
# data = loader.load_and_split()


# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(data)

# # Agrega el documento a la coleccion testing en la base de datos
# Chroma.from_documents(client=chroma_client, collection_name='Testing', documents=all_splits, embedding=local_embeddings)
# chroma_client.get_collection('Testing')


Retrieving:

In [5]:
# Genera el vectorstore desde la base de datos
vectorstore = Chroma(
    client=chroma_client,
    collection_name="Testing",
    embedding_function=local_embeddings,
)

In [6]:
model = ChatOllama(
    model="llama3",
)

In [7]:
vectorstore

In [8]:
# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

# Desired schema for response
class AnswerWithSources(TypedDict):
    """An answer to the question, with sources."""

    answer: str
    sources: Annotated[
        List[str],
        ...,
        "List of sources (author + year) used to answer the question",
    ]

# Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. "
    # "Use three sentences maximum"
    # "and keep the answer concise."
    "Try to answer in the language of the context"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


# This Runnable takes a dict with keys 'input' and 'context',
# formats them into a prompt, and generates a response.
rag_chain_from_docs = (
    {
        "input": lambda x: x["input"],  # input query
        "context": lambda x: format_docs(x["context"]),  # context
    }
    | prompt  # format query and context into prompt
    | model  # generate response
    | StrOutputParser()  # coerce to string
    # | model.with_structured_output(AnswerWithSources)
)

# Pass input query to retriever
retrieve_docs = (lambda x: x["input"]) | retriever

# Below, we chain `.assign` calls. This takes a dict and successively
# adds keys-- "context" and "answer"-- where the value for each key
# is determined by a Runnable. The Runnable operates on all existing
# keys in the dict.
chain = RunnablePassthrough.assign(context=retrieve_docs).assign(
    answer=rag_chain_from_docs
)

In [9]:
question = """

Decime el objetivo y resumí los puntos clave del sistema 2710

"""

response = chain.invoke({"language": "italian", "input":question })

print(response["answer"])
print('\n')
for i in response['context']:
    print(i.metadata)

El objetivo del Sistema 2710 es limitar la presión de la contención en caso de accidente severo, mediante el venteo a la atmósfera.

Los puntos clave del sistema son:

* El sistema cuenta con una línea de venteo que conecta la cámara de aire de la PS con la atmósfera.
* La línea de venteo tiene un orificio de restricción (2710-FO-10005) para limitar el caudal de venteo.
* Se utiliza argón para inertizar la línea de venteo y evitar el ingreso de oxígeno.
* El sistema cuenta con válvulas de bloqueo (VEM-001 y VEM-002) que se ubicaran lo más cerca posible de la contención.
* El sistema tiene la capacidad de muestrear la cámara de la PS y enviar los resultados al sistema 2600 para su monitoreo.
* El sistema puede actuar en Nivel 3B - Etapa 2 o en N4 de DenP, limitando la presión de la contención mediante el venteo a la atmósfera.


{'page': 14, 'source': 'MD-CAREM25XT-34-r0.pdf'}
{'page': 8, 'source': 'MD-CAREM25XT-34-r0.pdf'}
{'page': 10, 'source': 'MD-CAREM25XT-34-r0.pdf'}
{'page': 23, '

In [16]:
question = """

como es el control de temperatura del BR-001 del segundo sistema de extincion (SSE).



"""

response = chain.invoke({"input":question })

print(response["answer"])
print('\n')
for i in response['context']:
    print(i.metadata)

Según el contexto, en operación normal del reactor, el lazo de control TIC -10042 actúa sobre las 0700 -BQ-001 de manera on-off con una banda de 200 °C ±3 °C. Este lazo de control se habilita cuando la temperatura en el RPR sea superior a 240 °C.

En cuanto al segundo sistema de extinción (SSE), no hay información específica sobre el control de temperatura del BR-001 en ese contexto. Sin embargo, según el texto, cuando la temperatura del Reactor sea menor a 240 °C y el sistema 0700 se encuentre habilitado, el control TIC -10042 debe mantener el tanque acumulador a una temperatura entre 45 °C (banda inferior) y 40 °C (banda superior), inferior a la Temperatura del RPR con un control on-off de la resistencia y con un set point variable (deslizante).


{'page': 21, 'source': 'MD-CAREM25XT-37-r2.pdf'}
{'page': 3, 'source': 'MD-CAREM25XT-37-r2.pdf'}
{'page': 21, 'source': 'MD-CAREM25XT-37-r2.pdf'}
{'page': 15, 'source': 'MD-CAREM25XT-37-r2.pdf'}
{'page': 15, 'source': 'MD-CAREM25XT-37-r2.pd

In [18]:
question = """

Que son catproducts?

"""

response = chain.invoke({"input":question })

print(response["answer"])
print('\n')
for i in response['context']:
    print(i.metadata)

Según el contexto proporcionado, un "CATProduct" es el bloque de gestión de información en CATIA para estructuras de producto. Es el archivo que contiene la información de la estructura y puede contener otros CATProducts y/o CATParts. Cuando se trabaja con estructuras de producto en CATIA y se desea guardarlas en ENOVIA, hay dos modos de trabajo: Structure Exposed o Estructura Expuesta, y Document Exposed, donde el archivo CATProduct guardado en la base contiene los links a los documentos hijos.


{'page': 20, 'source': 'MA-CAREM25CO-6-r0.pdf'}
{'page': 20, 'source': 'MA-CAREM25CO-6-r0.pdf'}
{'page': 17, 'source': 'MA-CAREM25CO-6-r0.pdf'}
{'page': 23, 'source': 'MA-CAREM25CO-6-r0.pdf'}
{'page': 18, 'source': 'MA-CAREM25CO-6-r0.pdf'}
{'page': 4, 'source': 'DD-CAREM25CO-3-r0_b.pdf'}


In [11]:
question = """

Decime el objetivo y resumí los puntos clave del sistema 2800

"""

response = chain.invoke({"input":question })

print(response["answer"])
print('\n')
for i in response['context']:
    print(i.metadata)

Según el contexto proporcionado, el objetivo del Sistema 2800 es "remoción de la potencia extraída por el SSECR en los EPFM en los que se requiera la actuación del mismo". Esto indica que el objetivo principal del sistema es remover el calor residual producido por el SSECR (Sistema de Seguridad de Energía Reactora) en los EPFM (Equipos de Potencia y Fuerza Magnética).

En cuanto a los puntos clave del Sistema 2800, aquí te presento una breve reseña:

* El sistema tiene intercambiadores de calor para remover el calor residual producido por el SSECR.
* Incluye componentes como cañerías de suministro y descarga propias.
* Dispone de medidores locales de presión en las líneas de ingreso al 2800 -BI-001 I/II.
* Cuenta con válvulas manuales para regular el caudal sobre la línea de ingreso.
* Se pueden leer localmente caudal, presión y temperatura en las líneas de descarga.
* El sistema tiene una autonomía calculada para la reposición de agua de 95.5 horas.

Espero que esto te ayude a entende

### MultyQuery

In [12]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever


metadata_field_info = [
    AttributeInfo(
        name="page",
        description="The page where info resides",
        type="string",
    ),
    AttributeInfo(
        name="source",
        description="The file name or document name",
        type="string",
    ),
]

document_content_description = "Content"

In [13]:
retriever = SelfQueryRetriever.from_llm(
    model,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

In [14]:
retriever.invoke("Que hay en el source MD-CAREM25XT-12-r2.pdf")

OutputParserException: Parsing text
I'm ready to help!

To structure the user's query to match the request schema, I'll provide a markdown code snippet with a JSON object formatted according to the provided schema.

Please go ahead and provide the user's query, and I'll do my best to transform it into a structured request in the format:
```json
{
    "query": string,
    "filter": string
}
```
Let me know when you're ready!
 raised following error:
Got invalid JSON object. Error: Expecting value: line 2 column 14 (char 15)

### Multy-Query

In [74]:

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives 
    | model 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [ ]:
generate_queries.invoke({'question':'Que es PLM'})

In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve

retriever = vectorstore.as_retriever(search_kwargs={"k": 6})
question = "Que es PLM?"

retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

In [ ]:

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | model
    | StrOutputParser()
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
question = "Que es PLM?"

final_rag_chain.invoke({"question":question})

In [55]:
# Multi Query: Different Perspectives
template_perspectives = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
You should generate these versions in spanish
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template_perspectives)

generate_queries = (
    prompt_perspectives 
    | model 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve

retriever = vectorstore.as_retriever(search_kwargs={"k": 6})
retrieval_chain = generate_queries | retriever.map() | get_unique_union


# Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. "
    # "Use three sentences maximum"
    # "and keep the answer concise."
    "Try to answer in the language of the context"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)


# Incorporate the retriever into a question-answering chain.
template = (
    "Sos un assitente para contestar preguntas"
    "Usa el siguiente contexto para contestar"
    "la pregunta. Si no sabes la respuesta, decí no lo sé"
    "Contestá en español"
    
    "\n\n"
    "{context}"
    "\n\n"
    "Question: {question}"
)

prompt = ChatPromptTemplate.from_template(template)

# This Runnable takes a dict with keys 'input' and 'context',
# formats them into a prompt, and generates a response.

final_rag_chain = (
    {
        "context": retrieval_chain, 
        "question": itemgetter("question")
    }
    | prompt  # format query and context into prompt
    | model  # generate response
    | StrOutputParser()  # coerce to string
    # | model.with_structured_output(AnswerWithSources)
)

In [57]:

question = """

Que es CATIA y para que se usa en el proyecto CAREM25?

"""

response = final_rag_chain.invoke({"question":question })

print(response)

Según el contexto, CATIA V5 es un sistema de diseño y planificación de productos que se utiliza en el proyecto CAREM25. En particular, se utiliza para la gestión del ciclo de vida de los productos (Product Lifecycle Management o PLM) y permite la creación, revisión y seguimiento de modificaciones a través de una herramienta llamada "Action" dentro de ENOVIA V5.

En el contexto del proyecto CAREM25, CATIA V5 se utiliza para:

* Generar reportes
* Realizar control y seguimiento de modificaciones
* Crear representaciones 3D (CATPart), en sambles (CATProduct) y planos (CATDrawing)
* Monitoreo y trazabilidad a lo largo del ciclo de vida de los productos.

En general, CATIA V5 es una herramienta muy versátil que se utiliza para la diseño, análisis, simulación y creación de prototipos, y en este contexto específico, se utiliza en el proyecto CAREM25 para gestionar el ciclo de vida de los productos.


In [64]:
question = """

Decime el objetivo y resumí los puntos clave del sistema 2800

"""

response = final_rag_chain.invoke({"question":question })

print(response)

**Objetivo:** El sistema 2800 tiene como objetivo facilitar la conexión por medio de manguera a la pileta de sello de descarga del sistema terciario, proporcionando información fácilmente accesible para el operador.

**Puntos clave:**

1. El sistema dispone de medidores locales de presión, caudal y temperatura en las líneas de ingreso y descarga.
2. Cuenta con una válvula manual para regulación del caudal sobre la línea de ingreso.
3. Está vinculado a otros sistemas de procesos, incluyendo el sistema primario (Sistema 0100), el sistema de interconexión de recintos de la contención (Sistema 0470) y otros.

En general, el sistema 2800 se enfoca en monitorear y controlar los flujos y presiones en las líneas de ingreso y descarga, lo que es fundamental para la seguridad y eficiencia del proceso.


In [15]:
response

'Based on the provided context, it appears that the system 2800 is a system extension for the state secure system.\n\nThe objective of the system 2800 is not explicitly stated in the given context. However, based on the information provided, it can be inferred that the system 2800 is designed to provide certain features and functionalities related to pressure, temperature, and flow measurement, as well as valve control and monitoring.\n\nKey points about the system 2800:\n\n1. It has local pressure meters in the input lines (2800-BI-001 I/II) located outside the reactor building.\n2. It has a manual valve for regulating flow rate on the input line.\n3. It measures flow rate, pressure, and temperature locally at both input and output lines, with sensors located outside the reactor building.\n4. The system 2800 is designed to provide information that can be easily accessed by operators.\n\nPlease note that this summary is based on a limited amount of context provided, and there may be ad